<a href="https://colab.research.google.com/github/nyp-sit/it3103/blob/main/week7/word_embeddings_glove.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Using Pre-trained Word embeddings

In this lab exercise, we will use a pretrained word embedding for our text classification task, instead of training our own embedding layer. 

## Setup

In [ ]:
import io
import os
import re
import shutil
import string
import tensorflow as tf
import numpy as np

from datetime import datetime
import tensorflow as tf
from tensorflow.keras import Model, Sequential
from tensorflow.keras.layers import Activation, Dense, Embedding, GlobalAveragePooling1D, Flatten
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

### Download the IMDb Dataset

Download the dataset using Keras file utility and do the clean-up as before

In [ ]:
url = "https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"

dataset = tf.keras.utils.get_file("aclImdb_v1.tar.gz", url,
                                    untar=True, cache_dir='.',
                                    cache_subdir='')

dataset_dir = os.path.join(os.path.dirname(dataset), 'aclImdb')
train_dir = os.path.join(dataset_dir, 'train')

In [ ]:
remove_dir = os.path.join(train_dir, 'unsup')
shutil.rmtree(remove_dir)

Next, create a `tf.data.Dataset` using `tf.keras.preprocessing.text_dataset_from_directory`. 

Use the `train` directory to create both train and validation datasets with a split of 20% for validation.

In [ ]:
batch_size = 1024
seed = 123
train_ds = tf.keras.preprocessing.text_dataset_from_directory(
    'aclImdb/train', batch_size=batch_size, validation_split=0.2, 
    subset='training', seed=seed)
val_ds = tf.keras.preprocessing.text_dataset_from_directory(
    'aclImdb/train', batch_size=batch_size, validation_split=0.2, 
    subset='validation', seed=seed)

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

## Text preprocessing

We then initialize a TextVectorization layer with the desired parameters to vectorize our movie reviews.

In [ ]:
# Vocabulary size and number of words in a sequence.
vocab_size = 10000
sequence_length = 100

# Use the text vectorization layer to normalize, split, and map strings to 
# integers. Note that the layer uses the custom standardization defined above. 
# Set maximum_sequence length as all samples are not of the same length.
vectorize_layer = TextVectorization(
    max_tokens=vocab_size,
    output_mode='int',
    output_sequence_length=sequence_length)

# Make a text-only dataset (no labels) and call adapt to build the vocabulary.
text_ds = train_ds.map(lambda x, y: x)
vectorize_layer.adapt(text_ds)

## Getting the pre-trained embedding model

We will use the pre-trained GloVe embeddings available from [stanford site](https://nlp.stanford.edu/projects/glove/). The original zip file contains embedding of different dimensions (e.g. 50d, 100d, 200d, etc) and is more than 800MB in file size. To save downloading time, we have made a copy of 50d GloVe file available on our course website for download. If you want to experiment with other embedding dimensions, please feel free to download from the stanford site.


In [ ]:
glove_url = 'https://nyp-aicourse.s3-ap-southeast-1.amazonaws.com/pretrained-models/glove.6B.50d.zip'

glove_files = tf.keras.utils.get_file("glove.6B.50d.zip", glove_url,
                                    extract=True, cache_dir='.',
                                    cache_subdir='')

## Load the Embedding layer

If you examined the extracted files, you will notice filenames like glove.6B.100d, glove.6B.50d, etc.  The 100d and 50d represents the different embedding dimensions.  

We will try the 50d embedding first.

In the code below, we read the embeddings from the download file line by line to create the embeddings index and then initialize the Keras embedding layer with this embeddings index.



In [ ]:
# Load up the GloVe word embedding data

embedding_dim = 50

print("Loading GloVe Word Embedding...")
embeddings_index = {}
with open('glove.6B.50d.txt', encoding="utf8") as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
    f.close()

Let's print out the embedding for the word `happy`.

In [ ]:
embeddings_index['happy']

In [ ]:
# Construct the word embedding matrix that will be used in the 
# Embedding layer.

vocab = vectorize_layer.get_vocabulary()
glove_embedding_matrix = np.zeros((len(vocab), embedding_dim))
for i, word in enumerate(vocab):
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        glove_embedding_matrix[i] = embedding_vector

In [ ]:
vocab_size = len(vocab)
embedding_layer = Embedding(vocab_size, 
                            embedding_dim, 
                            input_length = sequence_length,
                            weights=[glove_embedding_matrix],  
                            trainable=False)

## Create a classification model

We will now create the model as before, but this time with embedding layer initialized with pretrained embedding.

In [ ]:
model = Sequential([
    vectorize_layer,
    embedding_layer,
    GlobalAveragePooling1D(),
    Dense(16, activation='relu'),
    Dense(8, activation='relu'),
    Dense(1, activation='sigmoid')
])

## Compile and train the model

In [ ]:
root_logdir = os.path.join(os.curdir, "tb_logs")

def get_run_logdir():    # use a new directory for each run
	import time
	run_id = time.strftime("run_%Y_%m_%d-%H_%M_%S")
	return os.path.join(root_logdir, run_id)

run_logdir = get_run_logdir()
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=run_logdir)

Compile and train the model using the `Adam` optimizer and `BinaryCrossentropy` loss. 

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
              metrics=['accuracy'])

In [ ]:
model.fit(
    train_ds,
    validation_data=val_ds, 
    epochs=15,
    callbacks=[tensorboard_callback])

In [ ]:
%load_ext tensorboard
%tensorboard --logdir tb_logs

With pretrained embedding layer, we reaches a validation accuracy of around 70%, worse than jointly train our embedding layer with the classification task.

If we have enough data (like in our case), joinly train our own embedding layer will usually yield a better performance.


Let's evaluate the model on our test dataset.

In [ ]:
test_ds = tf.keras.preprocessing.text_dataset_from_directory(
    'aclImdb/test', 
    batch_size=batch_size)

In [ ]:
model.evaluate(test_ds)